In [1]:
import os
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display

In [2]:
# special need for Google Colab
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/ColabNotebooks/DocDenoise")
!ls

Mounted at /content/drive
autoencoders.ipynb	 chineseInvoice_ae_sub.ipynb  data		zip
checkpoints		 chineseInvoice_cg.ipynb      images
chineseInvoice_ae.ipynb  cycleGAN_old.ipynb	      paddle_ocr.ipynb


In [3]:
# check GPU details
!nvidia-smi

Fri Jul  1 21:37:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install paddlepaddle-gpu
!pip install "paddleocr>=2.0.1"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 576.1 MB 10 kB/s 
     |████████████████████████████████| 373 kB 62.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 334 kB 6.9 MB/s 
     |████████████████████████████████| 604 kB 44.8 MB/s 
     |████████████████████████████████| 948 kB 54.8 MB/s 
     |████████████████████████████████| 2.8 MB 51.5 MB/s 
     |████████████████████████████████| 55.7 MB 92.5 MB/s 
     |████████████████████████████████| 50 kB 7.1 MB/s 
     |████████████████████████████████| 404 kB 68.5 MB/s 
     |████████████████████████████████| 204 kB 64.2 MB/s 
     |████████████████████████████████| 2.3 MB 53.2 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149867 sha256=9f4a78c2b04e251a934bffdc59213cfca6b80c8589a7

In [5]:
from paddleocr import PaddleOCR, draw_ocr
ocr = PaddleOCR(use_angle_cls=True, lang='ch') # need to run only once to download and load model into memory

/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import (exp, inf, pi, sqrt, floor, sin, cos, around, int,
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sc

download https://paddleocr.bj.bcebos.com/PP-OCRv3/chinese/ch_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer/ch_PP-OCRv3_det_infer.tar


100%|██████████| 3.83M/3.83M [00:09<00:00, 405kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv3/chinese/ch_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/ch/ch_PP-OCRv3_rec_infer/ch_PP-OCRv3_rec_infer.tar


100%|██████████| 11.9M/11.9M [00:14<00:00, 837kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 1.45M/1.45M [00:07<00:00, 200kiB/s]

[2022/07/01 21:39:49] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/root/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='f

In [11]:
input_path = 'data/processed/'
output_path = 'data/paddle_ocr_processed/'
input_images_name = sorted(os.listdir(input_path))
print(input_images_name)

['IMG_0834.JPG', 'IMG_0835.JPG', 'IMG_0836.JPG', 'IMG_0837.JPG', 'IMG_0838.JPG', 'IMG_0839.JPG', 'IMG_0840.JPG', 'IMG_0841.JPG', 'IMG_0842.JPG', 'IMG_0843.JPG', 'IMG_0844.JPG', 'IMG_0845.JPG', 'IMG_0846.JPG', 'IMG_0847.JPG', 'IMG_0848.JPG', 'IMG_0849.JPG', 'IMG_0850.JPG', 'IMG_0851.JPG', 'IMG_0852.JPG', 'IMG_0853.JPG', 'IMG_0854.JPG', 'IMG_0855.JPG', 'IMG_0856.JPG', 'IMG_0857.JPG', 'IMG_0858.JPG', 'IMG_0859.JPG', 'IMG_0860.JPG', 'IMG_0861.JPG', 'IMG_0862.JPG', 'IMG_0863.JPG', 'IMG_0864.JPG', 'IMG_0865.JPG', 'IMG_0866.JPG', 'IMG_0867.JPG', 'IMG_0868.JPG']


In [14]:
# Draw bounding boxes and text
def draw_image(path, bounds, color='red', width=2):
    myFont = ImageFont.truetype('data/font/msyh.ttc', 32)
    image = Image.open(path)
    image = image.rotate(90, expand=True) # uncomment this line if necessary
    draw = ImageDraw.Draw(image)
    for bound in bounds:
        p0, p1, p2, p3 = bound[0]
        draw.line([*p0, *p1, *p2, *p3, *p0], fill=color, width=width)
        draw.text(p3, bound[1][0], font=myFont, fill=(255, 0, 0))
    return image

In [15]:
# Save output images
for img_name in input_images_name[16:]:
    path = input_path + img_name
    result = ocr.ocr(path, cls=True)
    draw_image(path, result).save(output_path + img_name)
    print(path)

[2022/07/01 21:57:59] ppocr DEBUG: dt_boxes num : 90, elapse : 0.07921314239501953
[2022/07/01 21:57:59] ppocr DEBUG: cls num  : 90, elapse : 0.06569981575012207
[2022/07/01 21:57:59] ppocr DEBUG: rec_res num  : 90, elapse : 0.3105943202972412
data/processed/IMG_0850.JPG
[2022/07/01 21:58:00] ppocr DEBUG: dt_boxes num : 111, elapse : 0.08527493476867676
[2022/07/01 21:58:01] ppocr DEBUG: cls num  : 111, elapse : 0.0920257568359375
[2022/07/01 21:58:01] ppocr DEBUG: rec_res num  : 111, elapse : 0.37996792793273926
data/processed/IMG_0851.JPG
[2022/07/01 21:58:02] ppocr DEBUG: dt_boxes num : 74, elapse : 0.06806278228759766
[2022/07/01 21:58:02] ppocr DEBUG: cls num  : 74, elapse : 0.06568384170532227
[2022/07/01 21:58:02] ppocr DEBUG: rec_res num  : 74, elapse : 0.22760677337646484
data/processed/IMG_0852.JPG
[2022/07/01 21:58:04] ppocr DEBUG: dt_boxes num : 84, elapse : 0.07740592956542969
[2022/07/01 21:58:04] ppocr DEBUG: cls num  : 84, elapse : 0.06113719940185547
[2022/07/01 21:58:

In [ ]:
display(draw_image(input_path, result))

Output hidden; open in https://colab.research.google.com to view.